# 4.5 GBM(Gradient Boosting Machine)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

feature_name_df = pd.read_csv('./human_activity/features.txt', sep='\s+',
                             header=None, names=['column_index','column_name'])

feature_name = feature_name_df.iloc[:,1].values.tolist()

def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name','dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])
                                                                                             if x[1]>0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

pd.options.display.max_rows = 999
new_feature_name_df = get_new_feature_name_df(feature_name_df)

In [3]:
def get_human_dataset():
    
    # 각 데이터 파일들은 공백으로 분리되어 있으므로, read_csv에서 공백문자를 sep로 할당.
    feature_name_df = pd.read_csv('./human_activity/features.txt',sep='\s+',
                                 header=None, names=['column_index','column_name'])
    
    #중복된 feature명을 새롭게 수정하는 get_new_feature_name_df()를 이용하여 새로운 feature명 DataFrame생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    #DataFrame에 피처명을 컬럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name= new_feature_name_df.iloc[:,1].values.tolist()
    
    #학습 피처 데이터셋과 테스트 피처 데이터를 DataFrame으로 로딩. 컬럼명은 feature_name적용
    X_train = pd.read_csv('./human_activity/train/X_train.txt',sep='\s+', names=feature_name)
    X_test = pd.read_csv('./human_activity/test/X_test.txt',sep='\s+', names=feature_name)
    
    #학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩하고 컬럼명은 action으로 부여
    y_train = pd.read_csv('./human_activity/train/y_train.txt',sep='\s+',header=None, names=['action'])
    y_test = pd.read_csv('./human_activity/test/y_test.txt', sep='\s+',header=None,names=['action'])
    
    #로드된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state = 0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print(f'GBM 정확도 : {gb_accuracy:.4f}')
print(f'GBM 수행시간 : {time.time()-start_time:.1f}초')

GBM 정확도 : 0.9389
GBM 수행시간 : 537.6초


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state = 0)
gb_clf.fit(X_train, y_train)
#gb_pred = gb_clf.predict(X_test)
#gb_accuracy = accuracy_score(y_test, gb_pred)

#print(f'GBM 정확도 : {gb_accuracy:.4f}')
#print(f'GBM 수행시간 : {time.time()-start_time:.1f}초')

In [4]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100,500],
    'learning_rate':[0.05,0.1]
}
grid_cv = GridSearchCV(gb_clf, param_grid = params, cv=2, verbose = 1, n_jobs=-1)
grid_cv.fit(X_train, y_train)
print(f'최적 하이퍼 파라미터:\n {grid_cv.best_params_}')
print(f'최고 예측 정확도 :{grid_cv.best_score_:.4f}')

NameError: name 'gb_clf' is not defined

In [ ]:
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print(f'GBM 정확도:{gb_accuracy:.4f}')